# **ArcGIS Python API でオープンデータを検索する**

## はじめに

みなさん GIS のデータはどの様に手に入れていますか？

　最近はオープンデータも増え、そのデータも整備され使いやすくなってきていると思います。今回は GIS のデータの中でもいわゆるベクターデータに絞って話をしていきますが、例えば「GIS　オープンデータ」と検索すると以下の様ようなサイトが検索にヒットします。


 - [国土数値情報ダウンロードサイト（国土交通省）](https://nlftp.mlit.go.jp/ksj/)

 - [G空間情報センター（一般社団法人 社会基盤情報流通推進協議会）](https://front.geospatial.jp/)

 - [政府機関オープンデータポータル（esri Japan）](https://esrij-gov-japan.hub.arcgis.com/)


　Python で GIS を扱う場合、データをいちいちローカルにダウンロードしてどうのこうのというのが面倒になる場合も多くあると思います。私自身業務では ArcGIS ユーザーなので、ポータルサイトに自社のデータを保存しており、ArcGIS Python API でデータを取得して処理する事も多いのですが、ArcGIS ではアカウントを持っていない方でも ArcGIS Python API を使用すれば一般に公開されているデータを取得可能な事を、ArcGISユーザー以外はあまり知られていないような気がします。

　今回は ArcGIS Python API を使用して ArcGIS Online に一般公開されているデータを取得する方法を紹介したいと思います。ただし ArcGIS のアカウントが無ければ、実際に ArcGIS Online のコンテンツページを開いて検索するというような事は出来ない（多分？）と思いますので、検索も全て Python で行う事になります。「もっと詳しく見てみたいな」と思ったら ArcGIS を契約してみてもいいかもしれません。



ArcGIS Python API は pip でも conda でもインストール可能なので好きな方法でインストールしてください。

items:

- 全国都道府県界データ2022 ... {rows: 47, cols: 9}

- 全国市区町村界データ2022 ... {rows: 1906, cols: 16}

- 平成27年 国勢調査 町丁・字等界 ... {rows: 218100, cols: 20}

- 交通事故箇所（2022） ... {rows: 300819, cols: 58}

## **アイテムの検索方法**
アイテムの検索は `arcgis.GIS`のインスタンスを作成し `content.search`メソッドで行います。

In [1]:
import arcgis

gis = arcgis.GIS()

items = gis.content.search('title:都道府県界 2022')
items

[<Item title:"【防災クロスビュー】全国都道府県界データ2022" type:Feature Layer Collection owner:NIED_CRS@bosai>,
 <Item title:"全国都道府県界データ2022" type:Feature Layer Collection owner:Esri_JP_Content>]

アイテムがListに格納されているのでインデックスを指定して使用したいアイテムを取り出します。

In [2]:
item = items[1]
print(item)
item

<Item title:"全国都道府県界データ2022" type:Feature Layer Collection owner:Esri_JP_Content>


<Item title:"全国都道府県界データ2022" type:Feature Layer Collection owner:Esri_JP_Content>

In [4]:
item.accessInformation

'© Esri Japan'

次にアイテムの`layers`メソッドでLayerを取得します。

In [77]:
lyrs = item.layers
print(lyrs)

[<FeatureLayer url:"https://services.arcgis.com/wlVTGRSYTzAbjjiC/arcgis/rest/services/zenkokutodofukenkai/FeatureServer/0">]


これにはLayerが一つしかないようです。

`ArcGIS Python API`には geopandas のように GIS を pandas.DataFrame で扱う `Spatial Enebled DataFrame`が用意されています。アイテムから DataFrame を取り出す流れは以下の様になっています。

1. arcgis.gis.GISのインスタンスを作成し、`.content.search`メソッドでアイテム検索を行う。
    
    - Returns:  List[arcgis.gis.Item]

2. アイテムをListから取り出し、`.layers`メソッドでアイテム内の全てのLayerを取得。

    - Returns: List[arcgis.features.FeatureLayer]

3. LayerリストからLayerを取得

4. Layer から FeatureSet を取得（ Query はこの段階で行う）

    - Returns: arcgis.features.FeatureSet

5. FeatureSet から `.sdf` メソッドで Spatial Enabled DataFrame を取得

    - pandas.DataFrame

それでは Layer から DataFrame を取り出してみましょう。

In [94]:
lyr = lyrs[0]
feature_set = lyr.query()
sdf = feature_set.sdf

# 不要な列を知っているのでとりあえず消しておく

del_cols = [
    # 見やすい様に列を削除する。
    'OBJECTID',
    'Shape__Area',
    'Shape__Length',
    'SHAPE'
]

print(
    sdf
    .drop(del_cols, axis=1)
    .head()
    .to_markdown()
)

|    |   JCODE | KEN    | KEN_ENG   |            P_NUM |            H_NUM |
|---:|--------:|:-------|:----------|-----------------:|-----------------:|
|  0 |      01 | 北海道 | Hokkaido  |      5.22873e+06 |      2.79557e+06 |
|  1 |      02 | 青森県 | Aomori    |      1.26007e+06 | 594459           |
|  2 |      03 | 岩手県 | Iwate     |      1.2212e+06  | 530800           |
|  3 |      04 | 宮城県 | Miyagi    |      2.28211e+06 |      1.01661e+06 |
|  4 |      05 | 秋田県 | Akita     | 971604           | 425698           |


## **Queryの使い方①**

In [104]:
print(
    lyr
    .query(where="KEN = '青森県'")
    .sdf
    .drop(del_cols, axis=1)
    .head()
    .to_markdown()
)

|    |   JCODE | KEN    | KEN_ENG   |       P_NUM |   H_NUM |
|---:|--------:|:-------|:----------|------------:|--------:|
|  0 |      02 | 青森県 | Aomori    | 1.26007e+06 |  594459 |


## **全国市区町村界データの取得とGeometryを使用したQuery**

全国市区町村界のデータも公開されているので、ここから青森県内に含まれるデータを空間検索を使用してQueryしてみましょう。

今回は検索が面倒なので、直接 Layer の URL を指定します。

In [174]:
url = (
    'https://services.arcgis.com/wlVTGRSYTzAbjjiC/arcgis'
    '/rest/services/TrafficAccident_2022/FeatureServer/0'
)
lyr = arcgis.features.FeatureLayer(url)
lyr

<FeatureLayer url:"https://services.arcgis.com/wlVTGRSYTzAbjjiC/arcgis/rest/services/TrafficAccident_2022/FeatureServer/0">

In [185]:
geom = selected_sdf['SHAPE'].iloc[0]

In [189]:
from arcgis.geometry import filters
filter_ = filters.intersects(geom)
fset = lyr.query(geometry_filter=filter_)
sdf = fset.sdf

In [200]:
print(sdf.iloc[:5, 3: 10].to_markdown())

|    |   都道府県コード |   警察署等コード |   警察署等 |   事故内容 |   死者数 |   負傷者数 |   路線コード |
|---:|-----------------:|-----------------:|-----------:|-----------:|---------:|-----------:|-------------:|
|  0 |               20 |              102 |      20102 |          1 |        1 |          0 |        10150 |
|  1 |               20 |              102 |      20102 |          2 |        0 |          1 |        30000 |
|  2 |               20 |              102 |      20102 |          2 |        0 |          1 |        30000 |
|  3 |               20 |              103 |      20103 |          2 |        0 |          2 |        10500 |
|  4 |               20 |              109 |      20109 |          2 |        0 |          1 |        30000 |


## GeoJSONとして出力する

In [207]:
import json

fp = './traffic_accident.geojson'
with open(fp, mode='w') as file:
    fset_result = sdf.spatial.to_featureset()
    geojson_dict = json.loads(fset_result.to_geojson)
    json.dump(geojson_dict, file, indent=2)
